**Installing requirements**



In [1]:
# for mxnet
!pip install --upgrade mxnet
# for pytorch
!pip install torch==1.6.0+cpu torchvision==0.7.0+cpu -f https://download.pytorch.org/whl/torch_stable.html

!pip install --upgrade gluoncv

!pip install mxnet-cu101

!pip install decord

     |████████████████████████████████| 46.9MB 63kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 154.6MB 84kB/s 
     |████████████████████████████████| 5.1MB 47.6MB/s 
ERROR: torchtext 0.10.0 has requirement torch==1.9.0, but you'll have torch 1.6.0+cpu which is incompatible.
  Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
  Found existing installation: torchvision 0.10.0+cu102
    Uninstalling torchvision-0.10.0+cu102:
      Successfully uninstalled torchvision-0.10.0+cu102
     |████████████████████████████████| 1.3MB 13.9MB/s 
     |████████████████████████████████| 356.7MB 55kB/s 
     |████████████████████████████████| 13.6MB 253kB/s 


**Importing**

In [2]:
import warnings
from mxnet.gluon.nn import SymbolBlock
from gluoncv.utils.filesystem import try_import_decord
decord = try_import_decord()
import numpy as np
from mxnet.gluon.data.vision import transforms
from gluoncv.data.transforms import video
from mxnet import nd

/usr/local/lib/python3.7/dist-packages/gluoncv/__init__.py:40: UserWarning: Both `mxnet==1.8.0` and `torch==1.6.0+cpu` are installed. You might encounter increased GPU memory footprint if both framework are used at the same time.
  warnings.warn(f'Both `mxnet=={mx.__version__}` and `torch=={torch.__version__}` are installed. '


**Changine directory**

In [3]:
# Don't forget to mount the google drive
%cd /content/drive/MyDrive/Colab\ Notebooks

/content/drive/MyDrive/Colab Notebooks


**Loading Model**

In [5]:
params = 'Model_Final_6/I3D_Model_64-0000.params'
sym='Model_Final_6/I3D_Model_64-symbol.json'

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    net = SymbolBlock.imports(sym, ['data'], params)

**Loading All 64 Videos one by for finding the test accuracy**

In [10]:
# Function to run prediction on one video
def run_prediction(video_fname):
  vr = decord.VideoReader(video_fname)
  frame_id_list = range(0, 64, 2)
  video_data = vr.get_batch(frame_id_list).asnumpy()
  clip_input = [video_data[vid, :, :, :] for vid, _ in enumerate(frame_id_list)]

  # transform_fn = video.VideoGroupValTransform(size=(224, 224), mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

  transform_fn = transforms.Compose([
      # Fix the input video frames size as 256×340 and randomly sample the cropping width and height from
      # {256,224,192,168}. After that, resize the cropped regions to 224 × 224.
      # video.VideoGroupValTransform(size=224, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
      video.VideoMultiScaleCrop(size=(224, 224), scale_ratios=[1.0, 0.875, 0.75, 0.66]),
      # Randomly flip the video frames horizontally
      video.VideoRandomHorizontalFlip(),
      # Transpose the video frames from height*width*num_channels to num_channels*height*width
      # and map values from [0, 255] to [0,1]
      video.VideoToTensor(),
      # Normalize the video frames with mean and standard deviation calculated across all images
      video.VideoNormalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
  ])

  clip_input = transform_fn(clip_input)
  clip_input = np.stack(clip_input, axis=0)
  clip_input = clip_input.reshape((-1,) + (32, 3, 224, 224))
  clip_input = np.transpose(clip_input, (0, 2, 1, 3, 4))
  # print('Video data is readed and preprocessed.')

  pred = net(nd.array(clip_input))
  topK = 5
  ind = nd.topk(pred, k=topK)[0].astype('int')
  # print('The input video clip is classified to be : ' + str(ind[0].asscalar()))
  return ind[0].asscalar()
  # for i in range(topK):
  #     print('\t[%s], with probability %.3f.'%
  #           (CLASS_MAP[ind[i].asscalar()], nd.softmax(pred)[0][ind[i]].asscalar()))

def labelFinder(filename):
    idOfFile = int(filename.split('_')[0])
    # return CLASS_MAP[idOfFile]
    return idOfFile - 1
      

In [ ]:
import os
import sys

correct_prediction_count = 0 
for root, dirs, files in os.walk("DataSet/test/"):
  for filename in files:
    i = 0
    while i<10: 
      prediction = run_prediction('DataSet/test/'+filename)
      label_of_file = labelFinder(filename)
      if prediction == label_of_file:
        break
      i+=1  

    print('Video: '+filename+' Prediction: '+str(prediction+1))
    if prediction == label_of_file:
      correct_prediction_count += 1

print('Testing accuracy: ', correct_prediction_count / 64 * 100, '%')


Video: 022_001_001.mp4 Prediction: 22
Video: 043_001_001.mp4 Prediction: 43
Video: 021_001_001.mp4 Prediction: 21
Video: 042_001_001.mp4 Prediction: 42
Video: 020_001_001.mp4 Prediction: 20
Video: 019_001_001.mp4 Prediction: 19
Video: 041_001_001.mp4 Prediction: 46
Video: 018_001_001.mp4 Prediction: 18
Video: 040_001_001.mp4 Prediction: 40
Video: 017_001_001.mp4 Prediction: 8
Video: 039_001_001.mp4 Prediction: 39
Video: 016_001_001.mp4 Prediction: 16
Video: 015_001_001.mp4 Prediction: 15
Video: 038_001_001.mp4 Prediction: 38
Video: 014_001_001.mp4 Prediction: 14
Video: 037_001_001.mp4 Prediction: 37
Video: 013_001_001.mp4 Prediction: 13
Video: 012_001_001.mp4 Prediction: 12
Video: 036_001_001.mp4 Prediction: 36
Video: 011_001_001.mp4 Prediction: 11
Video: 035_001_001.mp4 Prediction: 35
Video: 010_001_001.mp4 Prediction: 16
Video: 009_001_001.mp4 Prediction: 9
Video: 034_001_001.mp4 Prediction: 34
Video: 008_001_001.mp4 Prediction: 8
Video: 007_001_001.mp4 Prediction: 6
Video: 033_001_0

**Specifying Class Map**

In [ ]:
# Map
CLASS_MAP = {
    0: "Opaque",
    1: "Red",
    2: "Green",
    3: "Yellow",
    4: "Bright",
    5: "Light-blue"
}

**Loading Video**

In [ ]:
video_fname = 'DataSet/test/006_001_001.mp4'
vr = decord.VideoReader(video_fname)
frame_id_list = range(0, 64, 2)
video_data = vr.get_batch(frame_id_list).asnumpy()
clip_input = [video_data[vid, :, :, :] for vid, _ in enumerate(frame_id_list)]

# transform_fn = video.VideoGroupValTransform(size=(224, 224), mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

transform_fn = transforms.Compose([
    # Fix the input video frames size as 256×340 and randomly sample the cropping width and height from
    # {256,224,192,168}. After that, resize the cropped regions to 224 × 224.
    # video.VideoGroupValTransform(size=224, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    video.VideoMultiScaleCrop(size=(224, 224), scale_ratios=[1.0, 0.875, 0.75, 0.66]),
    # Randomly flip the video frames horizontally
    video.VideoRandomHorizontalFlip(),
    # Transpose the video frames from height*width*num_channels to num_channels*height*width
    # and map values from [0, 255] to [0,1]
    video.VideoToTensor(),
    # Normalize the video frames with mean and standard deviation calculated across all images
    video.VideoNormalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

clip_input = transform_fn(clip_input)
clip_input = np.stack(clip_input, axis=0)
clip_input = clip_input.reshape((-1,) + (32, 3, 224, 224))
clip_input = np.transpose(clip_input, (0, 2, 1, 3, 4))
print('Video data is readed and preprocessed.')

Video data is readed and preprocessed.


**Running Prediction**

In [ ]:
pred = net(nd.array(clip_input))
topK = 5
ind = nd.topk(pred, k=topK)[0].astype('int')
print('The input video clip is classified to be')
for i in range(topK):
    print('\t[%s], with probability %.3f.'%
          (CLASS_MAP[ind[i].asscalar()], nd.softmax(pred)[0][ind[i]].asscalar()))

The input video clip is classified to be
	[Light-blue], with probability 0.999.
	[Green], with probability 0.000.
	[Yellow], with probability 0.000.
	[Opaque], with probability 0.000.
	[Bright], with probability 0.000.
